In [159]:
!pip install antigranular &> /dev/null

In [160]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: 279ef104-a0ec-4f75-bfad-42b40e719861, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [161]:
session.privacy_odometer()

{'total_epsilon_used': 0.0,
 'total_delta_used': 0.0,
 'library_costs': {},
 'dataset_costs': {}}

In [162]:
%%ag
x_train = transaction_fraud_hackathon_dataset["train_x"]
y_train = transaction_fraud_hackathon_dataset["train_y"]
x_test = transaction_fraud_hackathon_dataset["test_x"]

In [163]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from op_tensorflow import PrivateKerasModel, PrivateDataLoader

# Efficient Sequential model with privacy constraints in mind
seqM = Sequential([
    # Input block
    Dense(256 , input_shape=(9,)),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.3),

    # First dense block
    Dense(256),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.3),

    # Second dense block
    Dense(128),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.3),

    # Final refinement layer
    Dense(64),
    BatchNormalization(),
    tf.keras.layers.ReLU(),
    Dropout(0.3),

    # Output layer
    Dense(1, activation='sigmoid')
])

# Create DP keras model with strict privacy parameters
dp_model = PrivateKerasModel(
    model=seqM,
    l2_norm_clip=0.8,  # Reduced for privacy budget
    noise_multiplier=0.7  # Increased for stronger privacy guarantees
)

# Optimizer with conservative learning rate
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0005,  # Reduced learning rate
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
)

# Compile model
dp_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.10/site-packages/tensorflow/__init__.py:30: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils
2025-02-11 16:50:00.884429: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

2025-02-11 16:50:00.960010: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-11 16:50:01.070185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-11 16:50:01.070219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unabl

In [164]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train , label_df=y_train, batch_size=128)

In [165]:
%%ag
dp_model.fit(x=data_loader, epochs=40, target_delta=1e-05)

Epoch 1/40

274/274 - 21s - loss: 0.6299 - accuracy: 0.6491 - 21s/epoch - 78ms/step

Epoch 2/40

274/274 - 13s - loss: 0.6066 - accuracy: 0.6732 - 13s/epoch - 49ms/step

Epoch 3/40

274/274 - 14s - loss: 0.6013 - accuracy: 0.6768 - 14s/epoch - 50ms/step

Epoch 4/40

274/274 - 14s - loss: 0.5941 - accuracy: 0.6817 - 14s/epoch - 50ms/step

Epoch 5/40

274/274 - 13s - loss: 0.5892 - accuracy: 0.6890 - 13s/epoch - 49ms/step

Epoch 6/40

274/274 - 14s - loss: 0.5891 - accuracy: 0.6889 - 14s/epoch - 49ms/step

Epoch 7/40

274/274 - 14s - loss: 0.5899 - accuracy: 0.6864 - 14s/epoch - 50ms/step

Epoch 8/40

274/274 - 14s - loss: 0.5866 - accuracy: 0.6870 - 14s/epoch - 51ms/step

Epoch 9/40

274/274 - 14s - loss: 0.5880 - accuracy: 0.6857 - 14s/epoch - 51ms/step

Epoch 10/40

274/274 - 13s - loss: 0.5842 - accuracy: 0.6923 - 13s/epoch - 49ms/step

Epoch 11/40

274/274 - 14s - loss: 0.5830 - accuracy: 0.6926 - 14s/epoch - 50ms/step

Epoch 12/40

274/274 - 14s - loss: 0.5799 - accuracy: 0.6977 - 

In [166]:
%%ag
test=PrivateDataFrame(x_test).copy()
y_pred = dp_model.predict(test, label_columns=["output"])

469/469 [==============================] - 4s 7ms/step



In [167]:
%%ag
# Note that the predictions are a float scalar, so we scale it!
def f(x: float) -> float:
    return 1 if x > 0.5 else 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

In [168]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.7008597475841333, 'logs': {'BIN_ACC': 0.7008597475841333, 'LIN_EPS': -0.0}}

